In [22]:
# import dependencies
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import requests

# import API key
from config import api_key

# add citipy library
from citipy import citipy

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime


In [23]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=10)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10)

#the zip method combines the lats and lngs into a list of pairs
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [24]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


10

In [25]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key
#print(url)

In [28]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    #check to see if requests.get worked
   # print(requests.get(city_url).json())

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | mandalgovi
Processing Record 2 of Set 1 | ouegoa
Processing Record 3 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 4 of Set 1 | kamaishi
Processing Record 5 of Set 1 | lompoc
Processing Record 6 of Set 1 | afsin
Processing Record 7 of Set 1 | cape town
Processing Record 8 of Set 1 | bathsheba
Processing Record 9 of Set 1 | praia
Processing Record 10 of Set 1 | yulara
-----------------------------
Data Retrieval Complete      
-----------------------------


In [30]:
#create data frame of city weather data
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mandalgovi,45.7625,106.2708,57.43,65,59,10.76,MN,2022-07-28 22:31:23
1,Ouegoa,-20.3500,164.4333,74.05,70,22,10.04,NC,2022-07-28 22:31:24
2,Kamaishi,39.2667,141.8833,77.11,83,25,3.27,JP,2022-07-28 22:31:24
3,Lompoc,34.6391,-120.4579,87.01,73,0,14.97,US,2022-07-28 22:31:19
4,Afsin,38.2477,36.9140,66.87,25,0,2.42,TR,2022-07-28 22:31:24
5,Cape Town,-33.9258,18.4232,58.87,81,75,18.41,ZA,2022-07-28 22:30:35
6,Bathsheba,13.2167,-59.5167,83.01,84,20,17.27,BB,2022-07-28 22:29:29
7,Praia,14.9215,-23.5087,82.94,65,20,3.44,CV,2022-07-28 22:31:25
8,Yulara,-25.2406,130.9889,48.33,37,77,7.83,AU,2022-07-28 22:31:25


In [31]:
#reorder the data frame
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df [new_column_order]

In [32]:
#view reordered columns
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Mandalgovi,MN,2022-07-28 22:31:23,45.7625,106.2708,57.43,65,59,10.76
1,Ouegoa,NC,2022-07-28 22:31:24,-20.3500,164.4333,74.05,70,22,10.04
2,Kamaishi,JP,2022-07-28 22:31:24,39.2667,141.8833,77.11,83,25,3.27
3,Lompoc,US,2022-07-28 22:31:19,34.6391,-120.4579,87.01,73,0,14.97
4,Afsin,TR,2022-07-28 22:31:24,38.2477,36.9140,66.87,25,0,2.42
5,Cape Town,ZA,2022-07-28 22:30:35,-33.9258,18.4232,58.87,81,75,18.41
6,Bathsheba,BB,2022-07-28 22:29:29,13.2167,-59.5167,83.01,84,20,17.27
7,Praia,CV,2022-07-28 22:31:25,14.9215,-23.5087,82.94,65,20,3.44
8,Yulara,AU,2022-07-28 22:31:25,-25.2406,130.9889,48.33,37,77,7.83


In [34]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")